In [1]:
import sys
import subprocess
from pathlib import Path

# Project root
ROOT = Path("/Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty")

# Updated script name
SCRIPT = ROOT / "src" / "analysis" / "apply_prediction_models.py"   # adjust folder if needed

print("Running:", SCRIPT)

result = subprocess.run(
    ["python3", str(SCRIPT)],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE,
    text=True
)

print("===== STDOUT =====")
print(result.stdout)

print("===== STDERR =====")
print(result.stderr)


Running: /Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty/src/analysis/apply_prediction_models.py
===== STDOUT =====
Loading master list: /Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty/data/processed/master_list.csv

===== Processing position: WR =====
[INFO] Loading model from: /Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty/data/Bakery/_derived/WR/wr_model_seed12_subs20.pkl
[INFO] Added score column: bakery_WR_wr_model_seed12_subs20_seed12 (shape=(706,))
[INFO] Loading model from: /Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty/data/Bakery/_derived/WR/wr_model_seed12_subs30.pkl
[INFO] Added score column: bakery_WR_wr_model_seed12_subs30_seed12 (shape=(706,))
[INFO] Loading model from: /Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty/data/Bakery/_derived/WR/wr_model_seed12_subs40.pkl
[INFO] Added score column: bakery_WR_wr_model_seed12_subs40_seed12 (shape=(706,))
[INFO] Loading model from: /Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty/data/Bakery/_deriv

In [2]:
import pandas as pd
from pathlib import Path
import re

# ========= CONFIG =========
ROOT = Path("/Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty")
BAKERY_OUT_DIR = ROOT / "data" / "processed" / "bakery_outputs"

POSITIONS = ["WR", "RB", "TE", "QB"]


def find_player_col(columns):
    """Try to locate the player name column from common candidates."""
    candidates = ["player_name", "Player", "player", "Name", "Player Name"]
    lower_map = {c.lower(): c for c in columns}
    for cand in candidates:
        if cand.lower() in lower_map:
            return lower_map[cand.lower()]
    return None


for pos in POSITIONS:
    in_path = BAKERY_OUT_DIR / f"bakery_scores_{pos}.csv"
    if not in_path.exists():
        print(f"[WARN] No file found for {pos}: {in_path}")
        continue

    print(f"\n=== Processing {pos}: {in_path} ===")
    df = pd.read_csv(in_path)

    # Locate player column
    player_col = find_player_col(df.columns)
    if player_col is None:
        print(f"[WARN] No player column found in {in_path}. Skipping.")
        continue

    # Find ensemble columns
    ensemble_cols = [c for c in df.columns if re.match(r"^bakery_.*_ensemble$", c)]
    if not ensemble_cols:
        print(f"[WARN] No bakery_*_ensemble columns in {in_path}.")
        continue

    # --- Reorder: move ensemble col(s) right after player col ---
    cols = list(df.columns)
    for ec in ensemble_cols:
        cols.remove(ec)

    insert_at = cols.index(player_col) + 1
    for offset, ec in enumerate(ensemble_cols):
        cols.insert(insert_at + offset, ec)

    df = df[cols]

    # --- Sort rows by ensemble descending (first ensemble col is primary) ---
    df = df.sort_values(by=ensemble_cols, ascending=False).reset_index(drop=True)

    # Save back
    df.to_csv(in_path, index=False)
    print(f"[INFO] Updated and reordered file → {in_path}")

print("\n[DONE] Ensemble column reordering + sorting complete.")



=== Processing WR: /Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty/data/processed/bakery_outputs/bakery_scores_WR.csv ===
[INFO] Updated and reordered file → /Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty/data/processed/bakery_outputs/bakery_scores_WR.csv

=== Processing RB: /Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty/data/processed/bakery_outputs/bakery_scores_RB.csv ===
[INFO] Updated and reordered file → /Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty/data/processed/bakery_outputs/bakery_scores_RB.csv

=== Processing TE: /Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty/data/processed/bakery_outputs/bakery_scores_TE.csv ===
[INFO] Updated and reordered file → /Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty/data/processed/bakery_outputs/bakery_scores_TE.csv

=== Processing QB: /Users/chasesiegel/Desktop/Comp_Sci/Capstone/Dynasty/data/processed/bakery_outputs/bakery_scores_QB.csv ===
[INFO] Updated and reordered file → /Users/chasesiegel/Desktop/Comp_